# Advent of Code 2022: Day 15
https://adventofcode.com/2022/day/15


## Part 1
Find how many positions in a row that cannot contain a beacon

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Function to preprocess the data and return min/max values

In [2]:
def preprocess_data(data, return_max=True, return_min = True):
  
  # Setup the objects that
  # will potentially be returned.
  output_list = []
  max_X = 0
  max_Y = 0
  min_X = float('inf')
  min_Y = float('inf')
  

  # Go through each line
  # in the data.
  for item in data:
    
    # Setup a temporary
    # list, where the 
    # current item will go.
    temp_list = []
    
    # Split the item on ': closest beacon is at '.
    split = item.split(': closest beacon is at ')
    
    # Go through each item
    # that this split generated.
    for inner_item in split:
      
      # Setup a temporary list,
      # where the current inner
      # item will go.
      inner_temp_list = []
      
      # Split the inner item on ','.
      inner_split = inner_item.split(',')
      
      # Go through the two strings that 
      # this split generated. 
      i = 0
      for inner_inner_item in inner_split:
        
        # Setup an empty string
        curr = ""
        
        # Go through each character in the string
        for j in range(len(inner_inner_item)): 
          t = inner_inner_item[j]
          
          # If the current character is
          # a '-' or a digit, append it
          # to the string
          if t == '-' or t.isdigit():
            curr += t
          
          # If it is the last character of
          # the string, convert it to an 
          # integer. # Compare it to 
          # the current max and min values 
          # and update if needed.
          if j == (len(inner_inner_item)-1):
            curr = int(curr)
            if i == 0:
              if curr > max_X:
                max_X = curr
              if curr < min_X:
                min_X = curr
              i += 1
            elif i == 1:
                if curr > max_Y:
                  max_Y = curr
                if curr < min_Y:
                  min_Y = curr
                i = 0
        
            # Append the current number
            # to the inner list. 
            inner_temp_list.append(curr)
      
      # Append the inner list to
      # the temporary list.
      temp_list.append(inner_temp_list)
    
    # Append the temporary list
    # to the output list.
    output_list.append(temp_list)
  

  # Return the processed data, and 
  # potentially the minimum and 
  # maximum values. 
  if return_max == True and return_min == True:
    return output_list, max_X, max_Y, min_X, min_Y
  
  elif return_max == True:
    return output_list, max_X, max_Y
  
  elif return_min == True:
    return output_list, min_X, min_Y
  
  else:
    return output_list

### Function to mark a row

In [3]:
def mark_map(target_row, item, max_Y, test_row, min_X):
  # Get the position of the scanner
  # and the beacon. Shift the x
  # coordinate to the right by the
  # minimum X value. 
  s_x = item[0][0] + abs(min_X)
  s_y = item[0][1]
  b_x = item[1][0] + abs(min_X)
  b_y = item[1][1]

  # If the row should contain
  # a sensor or a beacon, mark it.
  if s_y== test_row:
      target_row[s_x] = "S"
  if b_y== test_row:
    target_row[b_x] = "B"

  # Get the distance from the 
  # sensor to the beacon
  dist = abs(s_x-b_x)+abs(s_y-b_y)

  # Go through the target row. Calculate
  # the distance to the sensor and 
  # mark with a '#' if the distance
  # is smaller than the distance
  # to the beacon. 
  for col in range(len(target_row)):
    curr_dist = abs(s_x-col) + abs(s_y-test_row)
    if curr_dist <= dist:
      if target_row[col] == '.':
        target_row[col] = "#"
  
  return target_row

### Function to mark a row for every item in the data

In [4]:
def mark_row(data,test_row, max_Y, min_X, target_row):
  for item in data:
    target_row = mark_map(target_row, item, max_Y, test_row, min_X)
  return target_row

### Function to check how many positions in a given row that cannot contain a beacon

In [5]:
def check_row(data,test_row):
  # Preprocess the data and get min/max values
  data_clean, max_X, max_Y, min_X, min_Y = preprocess_data(data)
  
  # Setup the target row, make it X*2 long for safety
  target_row = ['.' for y in range(max_X*2)]
  
  # Mark the row
  target_row = mark_row(data_clean, test_row, max_Y, min_X, target_row)

  # Count how many positions contain
  # a '#' or a 'S'
  pos_count = 0
  for col in target_row:
    if col == "#" or col == "S":
      pos_count += 1
  return pos_count

In [6]:
check_row(data_list, 2000000)

4424278

## Part 2
Find the only location that could have a beacon with x and y
coordinates between 0 and 4.000.000.

Solution by user: i_have_no_biscuits: 

https://www.reddit.com/r/adventofcode/comments/zmcn64/comment/j0b90nr/?utm_source=share&utm_medium=web2x&context=3


The shape of the area that the scanner covers can be described
  as a rhombus, i.e. a shape with
  4 sides. A side can be represented
  by a diagonal line. Going left to 
  right in the coordinate space, two
  of the lines will have a slope (gradient)
  of +1 
  
  y = x + sy-sx+r+1

  y = x + sy-sx-r-1

  and two will have a slope of -1

  y = -x + sx+sy+r+1

  y = -x + sx+sy-r-1

  where the sx and sy stand for the
  scanner positions. The user calls the second part of these equations, for example (sy-sx+r+1) for the first equation, the coefficients of the lines, while in fact they are the intercepts. Regardless, to find where two lines (x+a) and (-x+b) intercect, find the point ((b-a)/2 , (a+b)/2). One of these intercection points will be the only location where a beacon can exist, so calculate the pairwise intersection between all (x+a) lines and (-x+b) lines until one is found to be outside the radius of every scanner. 



In [7]:
def find_frequency(data, max_value):
  # Preprocess the data, no need
  # for any minimum or maxmimum values
  data_clean = preprocess_data(data, return_max=False, return_min=False)

  # Setup a dictionary which will store 
  # the radius of every sensor, using 
  # the sensors coordinates as the key.
  radius = {}
  for item in data_clean:
    s_x = item[0][0]
    s_y = item[0][1]
    b_x = item[1][0]
    b_y = item[1][1]
    radius[(s_x, s_y)] = abs(s_x-b_x)+abs(s_y-b_y)
  
  # Get the scanner coordinates
  scanners = radius.keys()

  # Setup sets to contain what the
  # user calls "coefficients" for 
  # the 4 lines. Based on the equations
  # they should probably be labeled 
  # as intercepts. 
  acoeffs, bcoeffs = set(), set()

  # Fill the sets with the "coefficients"
  for((x,y), r) in radius.items():
    acoeffs.add(y-x+r+1)
    acoeffs.add(y-x-r-1)
    bcoeffs.add(x+y+r+1)
    bcoeffs.add(x+y-r-1)
  
  # Check the intercetions
  # for each pair of a
  # and b lines. 
  for a in acoeffs:
    for b in bcoeffs:
      p = ((b-a)//2, (a+b)//2)
      if all(0<c<max_value for c in p):
        if all((abs(p[0]-t[0])+abs(p[1]-t[1]))>radius[t] for t in scanners):
          return (max_value*p[0])+p[1]

In [8]:
find_frequency(data_list, 4000000)

10382630753392